# Librairies


In [43]:
import numpy as np 
import matplotlib.pyplot as plt 
import time
import pyreadr
import xarray

# Préalable

## Initialisation

In [44]:
def initialize(ni,ri):
    Ui = np.zeros((ni, ri))
    indices = np.array_split(np.arange(ni), ri)
    for j, idx in enumerate(indices):
        Ui[idx, j] = 1 / np.sqrt(len(idx))

    return Ui


def initialize_G(r1,r2,r3):
    G= np.random.rand(r1,r2,r3)

    return G

## Calcul de l'erreur

In [45]:
def etalT(T):
    n1, n2, n3 = T.shape
    T_prime=np.transpose(T,(0,2,1)).reshape(n1,n3*n2)
    return T_prime

def U23G(U2, U3, G):
    G_=G[:,:,0]*U3[0,0]
    for i in range (1,U3.shape[1]):   
        G_=G_+G[:,:,i]*U3[0,i]
    U23=G_@U2.T

    for j in range (1, U3.shape[0]):
        G_=G[:,:,0]*U3[j,0]
        for i in range (1,U3.shape[1]):   
            G_=G_+G[:,:,i]*U3[j,i]
        U23=np.bmat([[U23 , G_@U2.T]])
    return U23

def calcul_error(T, G, U1, U2, U3):
    Mat=U1 @ U23G(U2, U3, G)
    T_prime=etalT(T)
    error=np.linalg.norm(T_prime-Mat)**2/np.linalg.norm(T_prime)**2
    return error

def calcul_e(T_,U,i,U23g):
    Mat=U @ U23g  
    t=T_[i,:]
    error=np.linalg.norm(t-Mat)**2/np.linalg.norm(t)**2
    return error

def U_normalize(U):
    m,n=U.shape
    for i in range(n):
        norm=np.linalg.norm(U[:,i])
        if norm > 0 :   
            U[:,i]=U[:,i]/norm
    

## Optimisation

In [46]:
def mode_n_product(X, M, mode):
    Xm = np.moveaxis(X, mode, 0)            
    Y = np.tensordot(M, Xm, axes=(1,0))      
    return np.moveaxis(Y, 0, mode) 

def optiG(T,U1,U2,U3,G,rcond=None):

    U_normalize(U1)
    U_normalize(U2)
    U_normalize(U3)
    
    pinv1 = U1.T
    pinv2 = U2.T
    pinv3 = U3.T
    G = mode_n_product(T, pinv1, mode=0)
    G = mode_n_product(G, pinv2, mode=1)
    G = mode_n_product(G, pinv3, mode=2)
    return G

In [47]:
def optiU(T, U, U2, U3, G):
    T_ = etalT(T)
    n1 = T.shape[0]
    r1 = G.shape[0]

    U_save = U.copy()
    value_save = calcul_error(T, G, U_save, U2, U3)

    U23g = U23G(U2, U3, G)  
    eps = 1e-12  

    for i in range(n1):

        best_j = 0
        best_err = float('inf')
        best_val = 0
        #Uiprev = U[i, :].copy()
        #err0 = calcul_e(T_, Uiprev, i, U23g)
        #print(f"i: {i}, err0: {err0}")
        for j in range(r1):

            #U_tmp = U[i,:].copy()
            #U_tmp[:] = 0
            Ulignei = np.zeros(r1)

            denom = np.linalg.norm(U23g[j, :])**2 + eps
            Ulignei[j] = (T_[i, :] @ U23g[j, :].T) / denom

            #U_test = U.copy()
            #U_test[i, :] = U_tmp

            err = calcul_e(T_, Ulignei, i, U23g)
            #print(f"i: {i}, j: {j}, err: {err}, best_err: {best_err}")
            if err < best_err:
                best_err = err
                best_val = Ulignei[j]
                best_j = j

        U[i, :] = 0
        #denom = np.linalg.norm(U23g[best_j, :])**2 + eps
        U[i, best_j] = best_val #(T_[i, :] @ U23g[best_j, :].T) / denom
        #Uiprev = U[i, :].copy()
        #err0 = calcul_e(T_, Uiprev, i, U23g)
        #print(f"i: {i}, err1: {err0}")

    err= calcul_error(T, G, U, U2, U3)
    return U, err

In [48]:
def Optimization(T, U1, U2, U3, G, max_iter=10,maxtime=300,tol=1e-6):
    erreurU1=[]
    erreurU2=[]
    erreurU3=[]
    start_time=time.time()

    for i in range (max_iter):
        print(f"__________itération {i+1} __________")

        T1 = T  
        #print(f"erreur avant : {calcul_error (T1,G,U1,U2,U3)}")
        U1,err = optiU(T1, U1, U2, U3, G)
        erreurU1.append(err)
        #print(f"Erreur après : {err}")

        T2 = np.transpose(T, (1, 0, 2))  
        #print(f"erreur avant : {calcul_error(T2,np.transpose(G, (1, 0, 2)),U2,U1,U3)}")
        U2,err = optiU(T2, U2, U1, U3, np.transpose(G, (1, 0, 2)))
        erreurU2.append(err)
        #print(f"Erreur U2 : {err}")

      
        T3 = np.transpose(T, (2, 0, 1))  
        #print(f"erreur avant : {calcul_error (T3,np.transpose(G, (2, 0, 1)),U3,U1,U2)}")
        U3,err = optiU(T3, U3, U1, U2, np.transpose(G, (2, 0, 1)))
        erreurU3.append(err)
        #print(f"Erreur U3 : {err}")
        print("Erreurs : ",erreurU1[-1], erreurU2[-1], erreurU3[-1])

        G=optiG(T,U1,U2,U3,G)

        error=calcul_error (T,G,U1,U2,U3)
        if erreurU1[-1]<tol or erreurU2[-1]<tol or erreurU3[-1]<tol:
            print("Seuil de tolérance atteint")
            break
        if time.time()-start_time>maxtime:
            print("Temps maximum écoulé")
            break
        
    return U1,U2,U3,G,erreurU1,erreurU2,erreurU3

## Print

In [49]:
def print_results(erreurU1,erreurU2,erreurU3):
    plt.plot(erreurU1, label='Erreur U1',color='red')
    plt.plot(erreurU2, label='Erreur U2',color='blue')
    plt.plot(erreurU3, label='Erreur U3',color='k')
    plt.yscale('log')
    plt.xlabel('Itérations')
    plt.ylabel('Erreur')
    plt.title('Convergence des erreurs')
    plt.legend()
    plt.show()

# Algo

Variables

In [50]:
r1=200
r2=200
r3=120
n1=250
n2=250
n3=150


U1=initialize(n1,r1)
U2=initialize(n2,r2)
U3=initialize(n3,r3)
G=initialize_G(r1,r2,r3)

T_test = mode_n_product(G, U1, mode=0)
T_test = mode_n_product(T_test, U2, mode=1)
T_test = mode_n_product(T_test, U3, mode=2)

if T_test.shape != (n1, n2, n3):
    raise ValueError("La forme de T_test est incorrecte.")

In [54]:
max_iter=50
tol=1e-6
maxtime=1500

In [52]:
np.random.seed(42)
T=T_test
n1,n2,n3=T.shape
U1=initialize(n1,r1)
U2=initialize(n2,r2)
U3=initialize(n3,r3)
G=initialize_G(r1,r2,r3)

In [ ]:
U1,U2,U3,G,erreurU1,erreurU2,erreurU3=Optimization(T, U1, U2, U3, G, max_iter,maxtime,tol)
print_results(erreurU1,erreurU2,erreurU3)

__________itération 1 __________


/var/folders/8b/44v24hrn61g51vz3tv9sgx1h0000gn/T/ipykernel_8064/1987577975.py:10: RuntimeWarning: divide by zero encountered in matmul
  U23=G_@U2.T
/var/folders/8b/44v24hrn61g51vz3tv9sgx1h0000gn/T/ipykernel_8064/1987577975.py:10: RuntimeWarning: overflow encountered in matmul
  U23=G_@U2.T
/var/folders/8b/44v24hrn61g51vz3tv9sgx1h0000gn/T/ipykernel_8064/1987577975.py:10: RuntimeWarning: invalid value encountered in matmul
  U23=G_@U2.T
/var/folders/8b/44v24hrn61g51vz3tv9sgx1h0000gn/T/ipykernel_8064/1987577975.py:16: RuntimeWarning: divide by zero encountered in matmul
  U23=np.bmat([[U23 , G_@U2.T]])
/var/folders/8b/44v24hrn61g51vz3tv9sgx1h0000gn/T/ipykernel_8064/1987577975.py:16: RuntimeWarning: overflow encountered in matmul
  U23=np.bmat([[U23 , G_@U2.T]])
/var/folders/8b/44v24hrn61g51vz3tv9sgx1h0000gn/T/ipykernel_8064/1987577975.py:16: RuntimeWarning: invalid value encountered in matmul
  U23=np.bmat([[U23 , G_@U2.T]])
/var/folders/8b/44v24hrn61g51vz3tv9sgx1h0000gn/T/ipykernel_8064

Erreurs :  0.2466606191273674 0.24666061911046083 0.24666061906574305
__________itération 2 __________
Erreurs :  0.24666061906471826 0.24666061906471778 0.24666061906469697
__________itération 3 __________
Erreurs :  0.24666061906469655 0.24666061906469672 0.2466606190646969
__________itération 4 __________
Erreurs :  0.24666061906469655 0.24666061906469655 0.2466606190646968
__________itération 5 __________
Erreurs :  0.24666061906469672 0.2466606190646969 0.2466606190646969
__________itération 6 __________
Erreurs :  0.24666061906469666 0.2466606190646969 0.2466606190646968
__________itération 7 __________
Erreurs :  0.24666061906469672 0.2466606190646968 0.24666061906469672
__________itération 8 __________
Erreurs :  0.24666061906469672 0.2466606190646968 0.2466606190646968
__________itération 9 __________
